In [ ]:
import numpy as np
import math
np.random.seed(1337)  # for reproducibility
import h5py
from sklearn.utils import class_weight

In [ ]:
from keras.models import Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Dense, Dropout, CNN2D, Activation
from keras.models import load_model
from keras import backend as K
from keras.layers import Flatten
import tensorflow as tf

# Read Input

Creating a rolling window of 299 bp long all along the test chromosome, the offset is one bp

In [ ]:
f = h5py.File('chr.hdf5','r')

nucleotid = np.array(f[f.keys()[0]])

f.close()

In [ ]:
X_one_hot = (np.arange(nucleotid.max()) == nucleotid[...,None]-1).astype(int)

In [ ]:
X_ = X_one_hot.reshape(X_one_hot.shape[0], X_one_hot.shape[1] * X_one_hot.shape[2])

In [ ]:
np.shape(X_)

In [ ]:
def rolling_window(array, window=(0,), asteps=None, wsteps=None, axes=None, toend=True):
        array = np.asarray(array)
        orig_shape = np.asarray(array.shape)
        window = np.atleast_1d(window).astype(int) # maybe crude to cast to int...
        
        if axes is not None:
            axes = np.atleast_1d(axes)
            w = np.zeros(array.ndim, dtype=int)
            for axis, size in zip(axes, window):
                w[axis] = size
            window = w
        
        # Check if window is legal:
        if window.ndim > 1:
            raise ValueError("`window` must be one-dimensional.")
        if np.any(window < 0):
            raise ValueError("All elements of `window` must be larger than 1.")
        if len(array.shape) < len(window):
            raise ValueError("`window` length must be less or equal `array` dimension.") 
    
        _asteps = np.ones_like(orig_shape)
        if asteps is not None:
            asteps = np.atleast_1d(asteps)
            if asteps.ndim != 1:
                raise ValueError("`asteps` must be either a scalar or one dimensional.")
            if len(asteps) > array.ndim:
                raise ValueError("`asteps` cannot be longer then the `array` dimension.")
            # does not enforce alignment, so that steps can be same as window too.
            _asteps[-len(asteps):] = asteps
            
            if np.any(asteps < 1):
                 raise ValueError("All elements of `asteps` must be larger then 1.")
        asteps = _asteps
        
        _wsteps = np.ones_like(window)
        if wsteps is not None:
            wsteps = np.atleast_1d(wsteps)
            if wsteps.shape != window.shape:
                raise ValueError("`wsteps` must have the same shape as `window`.")
            if np.any(wsteps < 0):
                 raise ValueError("All elements of `wsteps` must be larger then 0.")
    
            _wsteps[:] = wsteps
            _wsteps[window == 0] = 1 # make sure that steps are 1 for non-existing dims.
        wsteps = _wsteps
    
        # Check that the window would not be larger than the original:
        if np.any(orig_shape[-len(window):] < window * wsteps):
            raise ValueError("`window` * `wsteps` larger then `array` in at least one dimension.")
    
        new_shape = orig_shape # just renaming...
        
        # For calculating the new shape 0s must act like 1s:
        _window = window.copy()
        _window[_window==0] = 1
        
        new_shape[-len(window):] += wsteps - _window * wsteps
        new_shape = (new_shape + asteps - 1) // asteps
        # make sure the new_shape is at least 1 in any \"old\" dimension (ie. steps
        # is (too) large, but we do not care.
        new_shape[new_shape < 1] = 1
        shape = new_shape
        
        strides = np.asarray(array.strides)
        strides *= asteps
        new_strides = array.strides[-len(window):] * wsteps
        
        # The full new shape and strides:
        if toend:
            new_shape = np.concatenate((shape, window))
            new_strides = np.concatenate((strides, new_strides))
        else:
            _ = np.zeros_like(shape)
            _[-len(window):] = window
            _window = _.copy()
            _[-len(window):] = new_strides
            _new_strides = _
            
            new_shape = np.zeros(len(shape)*2, dtype=int)
            new_strides = np.zeros(len(shape)*2, dtypenucleotid=int)
            
            new_shape[::2] = shape
            new_strides[::2] = strides
            new_shape[1::2] = _window
            new_strides[1::2] = _new_strides
        
        new_strides = new_strides[new_shape != 0]
        new_shape = new_shape[new_shape != 0]
        
        return np.lib.stride_tricks.as_strided(array, shape=new_shape, strides=new_strides)

In [ ]:
wx = 299
X_slide = rolling_window(X_, window=(wx,4), asteps=None, wsteps=None, axes=None, toend=True)

In [ ]:
np.shape(X_slide)

In [ ]:
X_ = X_slide.reshape(X_slide.shape[0], X_slide.shape[2], X_slide.shape[3], 1)

In [ ]:
np.shape(X_)

# Test on Entire chromosome

In [ ]:
def MCC(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

In [ ]:
model = load_model('weights.hdf5',custom_objects={'MCC':MCC })

In [ ]:
pred = model.predict(X_)
np.save('pred_chr', pred)

# Analyses des résultats

Calculate the area under the mean of the zscore of the prediction in function of the distance to a TSS curve.

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
position = np.load('position_TSS_chr.npy')

In [ ]:
pred_chr = np.load('pred_chr.npy')

In [ ]:
X = stats.zscore(pred_chr)

In [ ]:
X.reshape(X.shape[0])

In [ ]:
length = 5000
matrix = X[position[0]-length:position[0]+length]

for i in position[1:] :
    window = X[i-length:i+length]
    window.reshape(2*length)
    matrix = np.concatenate((window,matrix),axis=0)

matrix = matrix.reshape(len(position),2*length)

In [ ]:
plt.figure()
plt.imshow(matrix,extent=(0,1,0,1))
plt.colorbar()
plt.title("myTitle")
plt.show()


In [ ]:
mean_matrix = stats.trim_mean(matrix, proportiontocut=0, axis=0)

In [ ]:
plt.plot(mean_matrix)
plt.ylabel('mean z_score')
plt.title("myTitle")

In [ ]:
area = trapz(mean_matrix[4000:6000])/600.
print("area =", area)